In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import json
import numpy as np
from functions import get_name

In [2]:
MODEL_PATH = ["mixedbread-ai/mxbai-embed-large-v1"]
DATA_PATH = ['../Data/AMC/aime_normal.json',\
             '../Data/AMC/amc12_normal.json',\
             '../Data/OlympiadBench_Dataset/data/OE_TO_maths_en_COMP.json', \
             '../Data/OlympiadBench_Dataset/data/TP_TO_maths_en_COMP.json'] # proof data, not for eval
OUT_PATH = '../Data/Embeddings'

In [3]:
for model_name in MODEL_PATH:
    model = SentenceTransformer(model_name)
    for data_path in DATA_PATH:
        with open(data_path, 'r') as file:
            data = json.load(file)
        output_questions = []
        output_context = []
        for input_dict in data:
            question = input_dict.get('question', '')
            solution_list = input_dict.get('solution', [])
            # TODO: take shortest sol as context now. need to revisit
            solution = min(solution_list, key=len)
            # TODO: now only encode question, test question + answer
            output_questions.append(question)
            output_context.append(f"{question} Solution: {solution}")
        embeddings = model.encode(output_questions)
        # embed_folder = create_next_model_folder(OUT_PATH)
        np.save(OUT_PATH + '/' + get_name(model_name) + "--" + get_name(data_path) +'.npy', embeddings)
        with open(OUT_PATH + '/' + get_name(data_path) + 'context.json', 'w') as f:
            json.dump(output_context, f)